In [ ]:
import cv2 as cv
from matplotlib import pyplot as plt
import imutils

import os
import sys
from pathlib import Path

import torch

from models.common import DetectMultiBackend
from utils.datasets import LoadImages
from utils.general import (check_file, check_img_size, increment_path, non_max_suppression, scale_coords, clip_coords,
                          xywh2xyxy, xyxy2xywh)
from utils.plots import Annotator, colors
from utils.torch_utils import select_device, time_sync

In [ ]:
half = False
augment=False
visualize=False
classes=None  
agnostic_nms=False 
save_img = True
save_crop = True
view_img = True
hide_labels = False
hide_conf = False

conf_thres=0.25  
iou_thres=0.45  
max_det=1000
line_thickness = 2

image_array = []

In [ ]:
def save_one_box2(xyxy, im, file='image.jpg', gain=1.02, pad=0, square=False, BGR=False, save=True):
    xyxy = torch.tensor(xyxy).view(-1, 4)
    boxes = xyxy2xywh(xyxy)
    if square:
        boxes[:, 2:] =boxes[:, 2:].max(1)[0].unsqueeze(1)
    boxes[:, 2:] = boxes[:, 2:] * gain + pad #box wh * gain + pad
    xyxy = xywh2xyxy(boxes).long()
    clip_coords(xyxy, im.shape)
    crop = im[int(xyxy[0, 1]):int(xyxy[0, 3]), int(xyxy[0, 0]):int(xyxy[0, 2]), ::(1 if BGR else -1)]
    
    return crop

In [ ]:
# Load Model
device = select_device('cpu')
weights = 'models/best.pt'

model = DetectMultiBackend(weights, device=device, dnn=False)
stride, names, pt, jit, onnx, engine = model.stride, model.names, model.pt, model.jit, model.onnx, model.engine
img_size = check_img_size(640, s=stride) #check image size

In [ ]:
source = ''
source = check_file(source)
save_dir = Path('')

# Half
half &= (pt or jit or engine) and device.type != 'cpu'
if pt or jit:
    model.model.half() if half else model.model.float()
    
dataset = LoadImages(source, img_size=img_size, stride=stride, auto=pt)
batch_size = 1
vid_path, vid_writer = [None] * batch_size, [None] * batch_size